In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('/content/drive/MyDrive/BOAZ_MINI_PROJECT_KAGGLE/train_ver2.csv/train_ver2.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = df.sort_values(by = ['ncodpers', 'fecha_dato'])

In [4]:
def lower_than(group,num):
    if len(group['fecha_dato']) < num:
        return None
    else :
        return group

In [5]:
df01 = df.groupby('ncodpers')
# groupby() 함수를 적용하면 입력한 기준을 바탕으로 pd.Series의 형태로 바뀌게 되는데, 이 때 만들어지는 group들을 이용해서 해당 group속의 데이터의 개수가 17개가 아니라면 None을 반환하고 17개,
# 즉 완전한 데이터를 갖춘 경우에만 return 하도록 한다.

In [ ]:
df01 = df01.apply(lower_than, num = 17)

In [ ]:
df01.head(3)

fecha_dato  ncodpers  ... ind_nom_pens_ult1 ind_recibo_ult1
ncodpers                                ...                                  
15889    416965   2015-01-28     15889  ...               0.0               0
         836224   2015-02-28     15889  ...               0.0               0
         1673961  2015-03-28     15889  ...               0.0               0

[3 rows x 48 columns]

In [ ]:
# 문자열 형태의 날짜 -> datetime 형태
df01['fecha_dato'] = pd.to_datetime(df01['fecha_dato'],format='%Y%m%d', errors='ignore')
df01.index = df01.index.droplevel(level=0)

Target_cols = df01.iloc[:1,].filter(regex="ind_+.*ult.*").columns.values
Feature_cols = [_ for _ in df01.iloc[:1,].columns.values if _ not in set(Target_cols)]
Feature_df = df01.loc[:,Feature_cols]
Target_df = df01.loc[:,Target_cols]

Obj_cols = df01.select_dtypes(include='object').columns



In [ ]:
Feature_df = pd.DataFrame(Feature_df, columns = Feature_df.columns)

In [ ]:
print(Target_df.shape, Feature_df.shape)

(10292888, 24) (10292888, 24)


In [ ]:
Feature_df.head(20)

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
416965,2015-01-28,15889,F,ES,V,56,1995-01-16,0.0,245,1.0,NaN,1,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,326124.9,01 - TOP
836224,2015-02-28,15889,F,ES,V,56,1995-01-16,0.0,245,1.0,NaN,1,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,326124.9,01 - TOP
1673961,2015-03-28,15889,F,ES,V,56,1995-01-16,0.0,245,1.0,NaN,1,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,326124.9,01 - TOP
1882060,2015-04-28,15889,F,ES,V,56,1995-01-16,0.0,245,1.0,NaN,1,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,326124.9,01 - TOP
3144382,2015-05-28,15889,F,ES,V,56,1995-01-16,0.0,245,1.0,NaN,1,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,326124.9,01 - TOP
3565837,2015-06-28,15889,F,ES,V,56,1995-01-16,0.0,245,1.0,NaN,1,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,326124.9,01 - TOP
4078996,2015-07-28,15889,F,ES,V,56,1995-01-16,0.0,245,1.0,NaN,1,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,326124.9,01 - TOP
5319234,2015-08-28,15889,F,ES,V,56,1995-01-16,0.0,246,1.0,NaN,1,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,326124.9,01 - TOP
5705188,2015-09-28,15889,F,ES,V,56,1995-01-16,0.0,247,1.0,NaN,1,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,326124.9,01 - TOP
6973298,2015-10-28,15889,F,ES,V,56,1995-01-16,0.0,248,1.0,NaN,1,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,326124.9,01 - TOP





```python
Feature_df = pd.get_dummies(Feature_df, prefix = Obj_col,drop_first=True)
```


```python
from sklearn.preprocessing import MinMaxScaler
Scaler = MinMaxScaler()
Scaled_df = Scaler.fit_transform(Feature_df)
```





#### 메모리의 사용을 줄이기 위해서 전날 미리 저장해 둔, 전처리와 Label, Feature 데이터로 나눠 놓은 dataframe을 이용해서 전처리 작업을 이어서 진행해 보고자 한다.

변수들이 너무 많을 때에는 **변수표**를 직접 작성하여서 실수를 줄이고 특성 간 관계를 파악하기가 쉽다!!!!!!  
**데이터 전처리 작업 혹은 EDA를 하기에 앞서 반드시 분석의 목적을 파악하고 그에 상응하는 준비 작업을 하자.**


1. 범주형 데이터를 숫자형 데이터로 바꾸어 주고자 한다.
  - 이를 위해서 sklearn의 Label Encoder을 사용해 보려 한다.

2. NULL값이 존재하는 column 제거
3. 수치형 변수의 경우 type바꾸어 주기


4. 월, 년도 정보 특성 추가
5. 1개월 전의 데이터를 Feature변수에 추가하기 (지난 달 데이터 변수 추가)
6. 신규 구매 건수만 추출
7. 추천될 Target 변수 특성 추가



In [4]:
import pandas as pd
import numpy as np
import sklearn

기존에 저장해 놓은, 17개의 데이터를 전부 가지고 있는 고객에대한 속성 정보를 저장해 놓은 csv 파일을 불러온다.
  - 확실히 이렇게 하니까 데이터를 불러오는데 드는 시간과 메모리가 절약이 된다.

In [5]:
Feature_df = pd.read_csv('/content/drive/MyDrive/BOAZ_MINI_PROJECT_KAGGLE/Feature_DF.csv')

In [6]:
Feature_df.isnull().sum()

Unnamed: 0                   0
fecha_dato                   0
ncodpers                     0
ind_empleado                 0
pais_residencia              0
sexo                         0
age                          0
ind_nuevo                    0
antiguedad                   0
indrel                       0
indrel_1mes                  0
tiprel_1mes                  0
indresi                      0
conyuemp                     0
indfall                      0
tipodom                      0
cod_prov                 13013
nomprov                      0
ind_actividad_cliente        0
renta                        0
segmento                     0
dtype: int64

In [7]:
Feature_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1620000 entries, 0 to 1619999
Data columns (total 21 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1620000 non-null  int64  
 1   fecha_dato             1620000 non-null  float64
 2   ncodpers               1620000 non-null  float64
 3   ind_empleado           1620000 non-null  float64
 4   pais_residencia        1620000 non-null  float64
 5   sexo                   1620000 non-null  float64
 6   age                    1620000 non-null  float64
 7   ind_nuevo              1620000 non-null  float64
 8   antiguedad             1620000 non-null  float64
 9   indrel                 1620000 non-null  float64
 10  indrel_1mes            1620000 non-null  float64
 11  tiprel_1mes            1620000 non-null  float64
 12  indresi                1620000 non-null  float64
 13  conyuemp               1620000 non-null  float64
 14  indfall           

In [11]:
Feature_df['age'] = pd.to_numeric(Feature_df['age'], errors = "coerce")
Feature_df.drop(columns = ['Unnamed: 0', 'fecha_alta','canal_entrada','ult_fec_cli_1t', 'indext'], axis=1, inplace=True)

- column중에 결측지 데이터가 있는 경우에는 Label Encoder의 encoding이 제대로 작동하지 않음이 확인가능했다.
- 범주형 변수 중에 결측치 존재 column:
  ```python 
  ['sexo', 'conyuemp', 'nomprov', 'segmento']
  ```
- 수치형 변수 중에 결측치 존재 column:
  ```python
  ['cod_prov', 'renta']
  ```
  

In [12]:
Obj_cols = Feature_df.select_dtypes(include='object').columns

In [5]:
Obj_cols

Index(['fecha_dato', 'ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes',
       'indresi', 'conyuemp', 'indfall', 'nomprov', 'segmento'],
      dtype='object')

In [13]:
for column in Obj_cols:
  # 여기서 na_sentinel의 값을 다른 정수형 변수로 설정해 주지 않는다면 NaN의 값을 drop하지 못하게 될 것이다.
  # 여기에서 어떠한 상수로 결측치를 바꿀 것인지는 직접 일일히 속성별로 따져 보는 것이 맞지 않나 싶다.
  # 그러나 일단은 그냥 우수 코드가 진행한 대로 -99라는 상수로 결측치를 처리해 보고자 한다.
  Encoded, Class = pd.factorize(Feature_df[column], na_sentinel = -99)
  Feature_df[column] = Encoded
  print(Feature_df[column])

0            0
1            1
2            2
3            3
4            4
            ..
10292883    12
10292884    13
10292885    14
10292886    15
10292887    16
Name: fecha_dato, Length: 10292888, dtype: int64
0           0
1           0
2           0
3           0
4           0
           ..
10292883    2
10292884    2
10292885    2
10292886    2
10292887    2
Name: ind_empleado, Length: 10292888, dtype: int64
0           0
1           0
2           0
3           0
4           0
           ..
10292883    0
10292884    0
10292885    0
10292886    0
10292887    0
Name: pais_residencia, Length: 10292888, dtype: int64
0           0
1           0
2           0
3           0
4           0
           ..
10292883    1
10292884    1
10292885    1
10292886    1
10292887    1
Name: sexo, Length: 10292888, dtype: int64
0           0
1           0
2           0
3           0
4           0
           ..
10292883    0
10292884    0
10292885    0
10292886    0
10292887    0
Name: tiprel_1mes, Len

In [14]:
# 수치형 변수의 결측치를 처리하고 데이터 타입을 변환한다.
Feature_df['age'].replace(' NA', -99, inplace = True)
Feature_df['age'] = Feature_df['age'].astype(np.int8)

Feature_df['renta'].replace('     NA', -99, inplace = True)
Feature_df['renta'].fillna(-99, inplace = True)
Feature_df['renta'] = Feature_df['renta'].astype(float).astype(np.int8)



In [24]:
Feature_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10292888 entries, 0 to 10292887
Data columns (total 20 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   fecha_dato             int64  
 1   ncodpers               int64  
 2   ind_empleado           int64  
 3   pais_residencia        int64  
 4   sexo                   int64  
 5   age                    int8   
 6   ind_nuevo              float64
 7   antiguedad             int64  
 8   indrel                 float64
 9   indrel_1mes            float64
 10  tiprel_1mes            int64  
 11  indresi                int64  
 12  conyuemp               int64  
 13  indfall                int64  
 14  tipodom                float64
 15  cod_prov               float64
 16  nomprov                int64  
 17  ind_actividad_cliente  float64
 18  renta                  int8   
 19  segmento               int64  
dtypes: float64(6), int64(12), int8(2)
memory usage: 1.4 GB


In [25]:
Feature_df.head(3)

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,ind_nuevo,antiguedad,indrel,indrel_1mes,tiprel_1mes,indresi,conyuemp,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
0,0,15889,0,0,0,56,0.0,245,1.0,1.0,0,0,0,0,1.0,28.0,0,1.0,-20,0
1,1,15889,0,0,0,56,0.0,245,1.0,1.0,0,0,0,0,1.0,28.0,0,1.0,-20,0
2,2,15889,0,0,0,56,0.0,245,1.0,1.0,0,0,0,0,1.0,28.0,0,1.0,-20,0


In [7]:
10292888/17

605464.0

- 총 605464명의 데이터를 가지고 있음이 확인이 가능하다.

In [15]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
Scaled_df = scaler.fit_transform(Feature_df)
Feature_df =pd.DataFrame(Scaled_df, columns = Feature_df.columns)

In [16]:
Feature_df.head(3)

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,ind_nuevo,antiguedad,indrel,indrel_1mes,tiprel_1mes,indresi,conyuemp,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
0,0.0000,0.0,0.0,0.0,0.99,0.469565,0.0,0.999989,0.0,0.0,0.0,0.0,0.99,0.0,0.0,0.529412,0.66,1.0,0.423529,0.980198
1,0.0625,0.0,0.0,0.0,0.99,0.469565,0.0,0.999989,0.0,0.0,0.0,0.0,0.99,0.0,0.0,0.529412,0.66,1.0,0.423529,0.980198
2,0.1250,0.0,0.0,0.0,0.99,0.469565,0.0,0.999989,0.0,0.0,0.0,0.0,0.99,0.0,0.0,0.529412,0.66,1.0,0.423529,0.980198


In [ ]:
Feature_df.to_csv('Feature_DF.csv')
Target_df.to_csv('Target_DF.csv')

In [17]:
import os
from pathlib import Path
# 이런 식으로 내가 원하는 대로 저장 경로의 directory를 바꾸어 준다.
# os가 결국에는 sys.
path = Path('/content/drive/MyDrive/BOAZ_MINI_PROJECT_KAGGLE')
os.chdir(path)

In [ ]:
Feature_df.to_csv('Feature_DF01.csv')

KeyboardInterrupt: ignored

- 이제 Feature_df에 있는 column별 데이터들을 정리할 때가 왔다.
- 각각의 속성별로 결측치를 보완해 주고, 변수형(수치형이 아닌) 데이터일 경우에 one-hot encoding을 진행해 주기보다는 (데이터의 용량이 너무 커짐) 일일히 dictionary를 만들어서 하나하나 수치형으로 바꾸어 준 뒤에 
```python
sklearn.preprocessing.MinMaxScaler()
```
을 이용해서 데이터의 범위를 0에서 1사이의 값으로 바꾸어 주고자 한다.

- 또한 만약에 머신 러닝 모델을 이용해서 학습을 하고자 한다면 데이터를 